In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the images as batches

In [2]:
img_nums = [0]#[n for n in range(0, 4900, 500)] #2750 #1234 #2314
imgs, gts = load_food_batch(img_nums)
num_imgs = len(imgs)
print(f"Total images: {num_imgs}")

Total images: 1


Check and filter for resolution

In [3]:
resol = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}

# Check for images with resolution over a certain threshold
thresh = 200_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels


In [4]:
# Filter out those images
imgs = {im_num: im for im_num, im in imgs.items() if resol[im_num] < thresh}
num_imgs = len(imgs)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 200,000: 1
   0: 196,608 pixels


Define parameters

In [5]:
# Which scribbles to use
bins = [0.05, 0.3, 1.5]
suff = ["w3"] #standing for width 3

# Which prediction models to use
convpaint_layers = [[0], [0,2]]
convpaint_scalings = [[1], [1,2], [1,2,4], [1,2,4,8]]
use_dino = True
use_ilastik = True

# Where to find and save the data
folder_path = "./FoodSeg103_results/data/food_run03b"

Loop and analyse

In [13]:
df_list = []

for img_num, img in imgs.items():
    ground_truth = gts[img_num]
    for bin in bins:
        for s in suff:
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            for layers in convpaint_layers:
                for scalings in convpaint_scalings:
                    print(f"ConvPaint: layers {layers}, scalings {scalings}")
                    pred_tag = generate_convpaint_tag(layers, scalings)
                    res = analyse_food_single_file(ground_truth, folder_path, img_num, mode="all", bin=bin, suff=s, pred_tag=pred_tag, show_res=False, image=None)
                    df_list.append(res)
            if use_ilastik:
                print("Ilastik")
                res = analyse_food_single_file(ground_truth, folder_path, img_num, mode="all", bin=bin, suff=s, pred_tag="ilastik", show_res=False, image=None)
                df_list.append(res)
            if use_dino:
                print("DINO")
                res = analyse_food_single_file(ground_truth, folder_path, img_num, mode="all", bin=bin, suff=s, pred_tag="dino", show_res=False, image=None)
                df_list.append(res)
            print("\n")

df = pd.concat(df_list, ignore_index=True)

IMG 0: bin 0.05, suff w3
ConvPaint: layers [0], scalings [1]
Class 1 | TP: 36906, FP: 6077, FN: 10364, TN: 143261
Class 1 | Precision: 0.859, Recall: 0.781, IoU: 0.692, F1: 0.818
Class 49 | TP: 44212, FP: 21319, FN: 2802, TN: 128275
Class 49 | Precision: 0.675, Recall: 0.940, IoU: 0.647, F1: 0.786
Class 67 | TP: 66835, FP: 10556, FN: 6392, TN: 112825
Class 67 | Precision: 0.864, Recall: 0.913, IoU: 0.798, F1: 0.887
Class 91 | TP: 9034, FP: 1669, FN: 20063, TN: 165842
Class 91 | Precision: 0.844, Recall: 0.310, IoU: 0.294, F1: 0.454
Overall Accuracy: 0.798
mPrecission: 0.810, mRecall: 0.736, mIoU: 0.608, mF1: 0.736
ConvPaint: layers [0], scalings [1, 2]
Class 1 | TP: 35025, FP: 2599, FN: 12245, TN: 146739
Class 1 | Precision: 0.931, Recall: 0.741, IoU: 0.702, F1: 0.825
Class 49 | TP: 44465, FP: 24721, FN: 2549, TN: 124873
Class 49 | Precision: 0.643, Recall: 0.946, IoU: 0.620, F1: 0.765
Class 67 | TP: 69907, FP: 12028, FN: 3320, TN: 111353
Class 67 | Precision: 0.853, Recall: 0.955, IoU

d:\Documents\Beruf & Ausbildung\Ausbildung\22-08 MSc BioInf\24FS_Master Thesis\02_scribbles_tests\image_analysis_helpers.py:12: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Class 1 | TP: 329, FP: 5, FN: 46941, TN: 149333
Class 1 | Precision: 0.985, Recall: 0.007, IoU: 0.007, F1: 0.014
Class 49 | TP: 43979, FP: 31093, FN: 3035, TN: 118501
Class 49 | Precision: 0.586, Recall: 0.935, IoU: 0.563, F1: 0.720
Class 67 | TP: 72613, FP: 47916, FN: 614, TN: 75465
Class 67 | Precision: 0.602, Recall: 0.992, IoU: 0.599, F1: 0.750
Class 91 | TP: 318, FP: 355, FN: 28779, TN: 167156
Class 91 | Precision: 0.473, Recall: 0.011, IoU: 0.011, F1: 0.021
Overall Accuracy: 0.596
mPrecission: 0.661, mRecall: 0.486, mIoU: 0.295, mF1: 0.376
ConvPaint: layers [0, 2], scalings [1, 2, 4, 8]
Class 1 | TP: 0, FP: 0, FN: 47270, TN: 149338
Class 1 | Precision: nan, Recall: 0.000, IoU: 0.000, F1: nan
Class 49 | TP: 43512, FP: 30860, FN: 3502, TN: 118734
Class 49 | Precision: 0.585, Recall: 0.926, IoU: 0.559, F1: 0.717
Class 67 | TP: 71524, FP: 50712, FN: 1703, TN: 72669
Class 67 | Precision: 0.585, Recall: 0.977, IoU: 0.577, F1: 0.732
Class 91 | TP: 0, FP: 0, FN: 29097, TN: 167511
Class 9

In [10]:
df.groupby(['prediction type'])[['accuracy']].mean()
df

,img_num,prediction type,scribbles mode,scribbles bin,suffix,max_class_pix_gt,min_class_pix_gt,pix_labelled,max_class_pix_labelled,min_class_pix_labelled,pix_in_img,perc. labelled,accuracy,mPrecision,mRecall,mIoU,mF1,scribbles,prediction
0,0,convpaint_l-0_s-1,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.798477,0.810239,0.736085,0.607542,0.736239,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
1,0,convpaint_l-0_s-1-2,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.793915,0.819503,0.717856,0.590825,0.713431,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
2,0,convpaint_l-0_s-1-2-4,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.616409,0.737228,0.514135,0.327133,0.430800,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
3,0,convpaint_l-0_s-1-2-4-8,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.592300,0.589094,0.482317,0.288452,0.731470,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
4,0,convpaint_l-0-2_s-1,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.654689,0.717723,0.563307,0.393737,0.525547,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
5,0,convpaint_l-0-2_s-1-2,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.632838,0.698033,0.526326,0.347441,0.456715,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
6,0,convpaint_l-0-2_s-1-2-4,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.596308,0.661454,0.486237,0.295054,0.376294,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
7,0,convpaint_l-0-2_s-1-2-4-8,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.585103,0.585095,0.475564,0.283960,0.724381,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_con...
8,0,ilastik,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.507690,0.677084,0.408109,0.306325,0.416179,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_ila...
9,0,dino,all,0.05,w3,73227,29097,349,196259,50,196608,0.177511,0.836177,0.800216,0.757936,0.653930,0.743069,./FoodSeg103_results/data/food_run03b/0000_scr...,./FoodSeg103_results/data/food_run03b/0000_din...


Save the data frame in a csv file:

In [ ]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
file_name = f"test_labels_vs_acc_{time_stamp}.csv"
df.to_csv(file_name, index=False)